In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# Révisions algorithmes sup

## Pivot de Gauss

In [4]:
from copy import deepcopy
import numpy as np

In [25]:
# test pour vérifier que la matrice est carrée
def testcarree(M):
    '''
    prend une liste de liste python
    et renvoie True si la matrice correspondante est carrée
    '''
    n = len(M)
    for i in range(n):
        if len(M[i]) != n:
            return False
    return True

def testcarree2(M):
    """
    verifie si un objet matrix de numpy est carré
    renvoie un booléen
    """
    n,m = M.shape
    return n == m


In [7]:
testcarree([[1,2,3],
            [4,5,6],
            [7,8,9]])

True

In [8]:
testcarree([[1,2,3],
            [4,5,6]])

False

In [26]:
M = np.matrix([[1,2,3],[4,5,6],[7,8,9]])
testcarree2(M)

True

In [27]:
M = np.matrix([[1,2,3],[4,5,6]])
testcarree2(M)

False

In [14]:
# Problème de la copie de matrices à 2 dimensions
# position du problème:
A = [[1,2,3], 
     [4,5,6], 
     [7,8,9]]

B = A
print(A)
print(B)
print(" ")
# Cela semble avoir fonctionné. Mais....
B[0][1] = 0
print(A)
print(B)
print(" ")
# Les deux objets n'en sont qu'un seul en réalité.
print(id(A))
print(id(B))
# Même adresse mémoire => B = A ne fait pas une copie de A!

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
 
[[1, 0, 3], [4, 5, 6], [7, 8, 9]]
[[1, 0, 3], [4, 5, 6], [7, 8, 9]]
 
140013019294768
140013019294768


In [16]:
def copieMat(A):
    return [ligne[:] for ligne in A]

def copieMat2(A):
    M = []
    for ligne in A:
        M.append(ligne[:])
    return M

def copieMat3(A):
    return deepcopy(A)

In [20]:
B = copieMat(A)
C = copieMat2(A)
D = copieMat3(A)
print(id(A),id(B),id(C),id(D),sep="\n")
print(" ")
# Adresses mémoire différentes donc objets différents!
B[0][0] = 10
C[0][1] = 11
D[0][2] = 12
print(A)
print(B)
print(C)
print(D)
# Il y a bien eu copie!

140013019294768
140013019375808
140013019436688
140013798063712
 
[[1, 0, 3], [4, 5, 6], [7, 8, 9]]
[[10, 0, 3], [4, 5, 6], [7, 8, 9]]
[[1, 11, 3], [4, 5, 6], [7, 8, 9]]
[[1, 0, 12], [4, 5, 6], [7, 8, 9]]


On travaille sur la matrice augmentée du vecteur Y (ou de l'identité) afin d'appliquer les mêmes transformations aux deux objets.

In [29]:
def augmente(M, Y):
    """
    renvoie la matrice augmentee
    avec M et Y des listes de listes
    """
    assert len(M) == len(Y)
    n = len(M)
    M2 = deepcopy(M)
    for i in range(n):
        M2[i] = M2[i] + Y[i] # concaténation
    return M2

M=[[1,2,3],[4,5,6],[7,8,9]]
Y=[[10],[11],[12]] # vecteur colonne
M2 = augmente(M,Y)
print(M,Y,M2,sep='\n')

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[10], [11], [12]]
[[1, 2, 3, 10], [4, 5, 6, 11], [7, 8, 9, 12]]


In [30]:
def affiche(M):
    for i in M:
        for j in i:
            print("{:.2f}".format(j),end="\t")
        print("")
    print("")

affiche(M2)
affiche(M)

1.00	2.00	3.00	10.00	
4.00	5.00	6.00	11.00	
7.00	8.00	9.00	12.00	

1.00	2.00	3.00	
4.00	5.00	6.00	
7.00	8.00	9.00	



### 1. recherche du plus grand pivot dans la colonne i

In [31]:
def indicePivot(A, i):
    """
    prend une liste de liste A
    et un entier i en arguments
    Renvoie l'indice du plus grand pivot
    en valeur absolue dans la colonne i dans ou en
    dessous de la diagonale.
    """
    n = len(A)
    imax = i
    for j in range(i+1, n):
        if abs(A[j][i]) > abs(A[imax][i]):
            imax = j
    return imax

### 2. Opérations nécessaires sur les matrices

In [32]:
def echange_lignes(A,i,j): 
    """
    echange les deux lignes L_i et L_j
    de la liste de listes passée en argument
    transformation en place,renvoie None
    """
    A[i],A[j]=A[j][:],A[i][:]
    return


def echange_lignes2(A,i,j):
    """
    echange les deux lignes L_i et L_j
    de la liste de listes passée en argument
    transformation en place,renvoie None
    """ 
    temp=A[i][:]
    A[i]=A[j][:]
    A[j]=temp
    return
 
def echange_lignes3(A,i,j):
    """
    echange les deux lignes L_i et L_j
    sur une copie de la liste de listes
    passée en argument
    Renvie la copie après transformation
    """
    B=deepcopy(A)
    B[i]=A[j][:]
    B[j]=A[i][:]
    return B
    
affiche(M2)
echange_lignes(M2,0,2) # echange la première et la troisième ligne
affiche(M2)

1.00	2.00	3.00	10.00	
4.00	5.00	6.00	11.00	
7.00	8.00	9.00	12.00	

7.00	8.00	9.00	12.00	
4.00	5.00	6.00	11.00	
1.00	2.00	3.00	10.00	



In [33]:
def transvection(A,i,j,mu): 
    """
    Applique la transformation
    L_i <- L_i+mu L_j
    en place et renvoie None
    """
    n = len(A[i])
    for k in range(n):
        A[i][k]+=mu*A[j][k]
    return

affiche(M2)
transvection(M2,1,0,-4/7)
affiche(M2)
transvection(M2,2,0,-1/7)
affiche(M2)

7.00	8.00	9.00	12.00	
4.00	5.00	6.00	11.00	
1.00	2.00	3.00	10.00	

7.00	8.00	9.00	12.00	
0.00	0.43	0.86	4.14	
1.00	2.00	3.00	10.00	

7.00	8.00	9.00	12.00	
0.00	0.43	0.86	4.14	
0.00	0.86	1.71	8.29	



In [34]:
def dilatation(A,i,mu): 
    """
    Applique la transformation
    L_i <- mu L_i
    en place et renvoie None
    """
    transvection(A,i,i,mu-1)
    return
    
affiche(M2)
dilatation(M2,0,1/7)
affiche(M2)

7.00	8.00	9.00	12.00	
0.00	0.43	0.86	4.14	
0.00	0.86	1.71	8.29	

1.00	1.14	1.29	1.71	
0.00	0.43	0.86	4.14	
0.00	0.86	1.71	8.29	



In [36]:
def I(n):
    """
    renvoie la matrice identité d'ordre n
    """
    rep=[]
    for i in range(n):
        ligne=[]
        for j in range(n):
            if i==j:
                ligne.append(1)
            else:
                ligne.append(0)
        rep.append(ligne)
    return rep

affiche(I(5))

1.00	0.00	0.00	0.00	0.00	
0.00	1.00	0.00	0.00	0.00	
0.00	0.00	1.00	0.00	0.00	
0.00	0.00	0.00	1.00	0.00	
0.00	0.00	0.00	0.00	1.00	



### 3. Pivot de gauss

In [40]:
def gauss(A0,Y0): 
    """
    renvoie le vecteur colonne X tel que AX=Y,
    on suppose qu'il existe.
    Vous pouvez décommenter les affichages afin de suivre
    la progression de l'algorithme
    """
    assert testcarree(A0)
    assert len(A0) == len(Y0)
    #affiche(A0)
    A=augmente(A0,Y0)
    #affiche(A)
    n = len(A)
    for i in range(n):
        j=indicePivot(A,i)
        if j!=i:
            echange_lignes(A,i,j)
            #affiche(A)
        for k in range(i+1,n):
            mu=-A[k][i]/A[i][i]
            transvection(A,k,i,mu)
            #affiche(A) 
    
    #si tout va bien, A est maintenant triangulaire supérieure, on peut aborder la remontée.
    for i in range(n):
        for j in range(i):
            mu=-A[j][i]/A[i][i]
            transvection(A,j,i,mu)
            #affiche(A)
    #A est maintenant diagonale
    for i in range(n):
        dilatation(A,i,1/A[i][i])
    
    #On n'a plus qu'à lire le vecteur
    X=[A[i][n:] for i in range(n)]
    return X
    
A=[[1,1,1],[1,-1,2],[2,-1,1]]
Y=[[2],[9],[7]]
print("Y")
affiche(Y)
print("= A")
affiche(A)
print("X")
X=gauss(A,Y)
affiche(X)

Y
2.00	
9.00	
7.00	

= A
1.00	1.00	1.00	
1.00	-1.00	2.00	
2.00	-1.00	1.00	

X
1.00	
-2.00	
3.00	



### 4. On peut s'en servie pour inverser une matrice

In [44]:
#inverser matrice
A=[[1,1,1],[1,-1,2],[2,-1,1]]
print("A")
affiche(A)
X=gauss(A,I(3))
print("A^-1")
affiche(X)

#et avec numpy
import numpy as np
A2=np.matrix(A)
print(A2**-1)

A
1.00	1.00	1.00	
1.00	-1.00	2.00	
2.00	-1.00	1.00	

A^-1
0.20	-0.40	0.60	
0.60	-0.20	-0.20	
0.20	0.60	-0.40	

[[ 0.2 -0.4  0.6]
 [ 0.6 -0.2 -0.2]
 [ 0.2  0.6 -0.4]]


### 5. Résolution d'un système de cramer

In [45]:
#resolution d'un systeme de Cramer
A=[[2,2,-3],[-2,-1,-3],[6,4,4]]
Y=[[2],[-5],[16]]
X=gauss(A,Y)
affiche(X)

#et avec numpy
A2=np.matrix(A)
Y2=np.array(Y)
X2 = A2**-1*Y2
print(X2)

-14.00	
21.00	
4.00	

[[-14.]
 [ 21.]
 [  4.]]


### 6. Cas problématiques

In [47]:
#pas de solution... Mais à cause des erreurs d'approximations,
# une sortie est tout de même obtenue !
A=[[0.1,0.1,0.1],[0.3,0.2,0.1],[0.4,0.3,0.2]]
Y=[[2],[9],[7]]
X=gauss(A,Y)
affiche(X)

# Même avec numpy!
A2=np.matrix(A)
Y2=np.array(Y)
print(A2**-1*Y)

-32025597350190172.00	
64051194700380376.00	
-32025597350190192.00	

[[-4.1175768e+16]
 [ 8.2351536e+16]
 [-4.1175768e+16]]


In [48]:
# Ici il y a une solution
# mais le code suivant produit une erreur
# à cause des arrondis
A=[[1,1+10.**(-15),1],[1-10**(-15),1,2],[0,0,1]]
Y=[[1],[0],[0]]
affiche(gauss(A,Y)) 

ZeroDivisionError: float division by zero

## Recherche dichotomique dans un tableau trié

In [50]:
def val(L,n):
    a=0
    b=len(L)
    while (b-a)>1:
        m=(a+b)//2
        if n<L[m]:
            b=m
        else:
            a=m
    return(L[a]==n)
    
# version récursive
def valrec(L, v):
    n = len(L)
    if n == 1: # Cas de base
        return(L[0] == v)
    else:
        m = n // 2
        if v < L[m]:
            return valrec(L[:m], v)
        else:
            return valrec(L[m:], v)

from random import randint
L=[randint(1,1000) for i in range(100)]
L.sort()
print(val(L,500))
print(L)

True
[2, 10, 23, 24, 48, 65, 65, 75, 88, 108, 111, 117, 143, 144, 147, 158, 158, 164, 170, 175, 179, 189, 212, 223, 233, 236, 244, 247, 253, 271, 324, 336, 347, 356, 374, 374, 390, 405, 407, 417, 436, 444, 453, 462, 467, 469, 471, 474, 480, 481, 483, 489, 491, 495, 500, 501, 501, 511, 545, 559, 563, 570, 582, 583, 590, 607, 632, 632, 652, 654, 679, 696, 698, 706, 711, 711, 713, 714, 740, 767, 781, 786, 801, 808, 833, 844, 854, 863, 865, 886, 912, 914, 916, 920, 934, 935, 940, 970, 982, 987]


## Recherche de zéro de fonction par dichotomie

In [52]:
def f(x):
    return(x**3+2*x**2-1)

a=0
b=1
epsilon=1e-8


def dicho(f,a,b,epsilon):
    assert f(a)*f(b)<0, "pas sur de trouver un zéro"
    m = (a+b)/2
    n = 0
    while abs(f(m))>epsilon:
        n+=1
        if f(a)*f(m)>0:
            a = m
        else:
            b = m
        m = (a+b)/2
    return(m,n) #renvoie aussi le nombre de tours de boucle nécessaires
x0,n=dicho(f,a,b,epsilon)
print("dichotomie")
print("x0 =",x0)
print("tours:",n)
print("f(x0) =",f(x0))

dichotomie
x0 = 0.6180339902639389
tours: 25
f(x0) = 5.47786283000562e-09


## Zéro de fonction par la méthode de Newton

In [59]:
# Il faut donner un x0
# On approxime la fonction par sa tangente en x0
# afin de trouver un meilleur zéro
# puis on itère.
def newton(f,x0,epsilon):
    def derivee(g,val):
        return((g(val+1e-6)-g(val))/1e-6) # h = 1e-6
    n=0
    while abs(f(x0))>epsilon:
        n+=1
        x0 = x0 - f(x0)/derivee(f,x0)
        if n >= 100:
            return "diverge"
    return(x0,n)

x=0.5
x0,n = newton(f,x,epsilon)
print("newton")
print("x0 =",x0)
print("tours:",n)
print("f(x0) =",f(x0))

print(" ")
def g(x):
    return (x**2-4)
# Si la dérivée en x0 est quasi nulle,
# le point suivant est renvoyé loin et cela fait
# perdre de l'efficacité à la méthode
# Si la dérivée en x0 est nulle,
# Nous obtenons un message d'erreur
x=1e-3
x0,n = newton(g,x,epsilon)
print("newton")
print("x0 =",x0)
print("tours:",n)
print("g(x0) =",g(x0))

newton
x0 = 0.61803398875005
tours: 4
f(x0) = 5.613287612504791e-13
 
newton
x0 = 2.0000000000000995
tours: 15
g(x0) = 3.979039320256561e-13


## Zéro de fonction par la méthode de lagrange

In [62]:
# il faut une fonction continue et deux points a et b
# tels que f(a) et f(b) soient de signe différent
# On approxime la fonction par la droite passant par
# (a,f(a)) et (b,f(b))
# Le x0 obtenu remplace soit a soit b en fonction du signe
# de f(x0) pour conserver l'hypothèse précédente. 
def lagrange(f,a,b,epsilon):
    assert f(a)*f(b)<0, "pas sur de trouver un zéro"
    m = (a*f(b)-b*f(a))/(f(b)-f(a))
    n = 0
    while abs(f(m))>epsilon:
        n+=1
        if f(a)*f(m)>0:
            a = m
        else:
            b = m
        m = (a*f(b)-b*f(a))/(f(b)-f(a))
    return(m,n)

def f(x):
    return(x**3+2*x**2-1)

a = 0
b = 1
epsilon = 1e-8

x0,n=lagrange(f,a,b,epsilon)
print("lagrange")
print("x0 =",x0)
print("tours:",n)
print("f(x0) =",f(x0))

lagrange
x0 = 0.6180339861914395
tours: 16
f(x0) = -9.256578503702428e-09
